In [1]:
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from features import speech_feat_extract
import numpy as np
import pickle

sess = ['Session'+str(i+1) for i in range(5)]

data = speech_feat_extract(sess, '../../../../HBA/IEMOCAP_full_release/')

4936


In [ ]:
check = False
if check:
    tr_y = []
    te_y = []
    for i in list(X['cat']['train']):
        if tuple(i) == (1,0,0,0):
            tr_y.append(1)
        if tuple(i) == (0,1,0,0):
            tr_y.append(2)
        if tuple(i) == (0,0,1,0):
            tr_y.append(3)
        if tuple(i) == (0,0,0,1):
            tr_y.append(4)
    for i in list(X['cat']['test']):
        if tuple(i) == (1,0,0,0):
            te_y.append(1)
        if tuple(i) == (0,1,0,0):
            te_y.append(2)
        if tuple(i) == (0,0,1,0):
            te_y.append(3)
        if tuple(i) == (0,0,0,1):
            te_y.append(4)
    tr_yc = np.array(tr_y)
    te_yc = np.array(te_y)

In [10]:
from keras import backend as K
import tensorflow as tf
'''
Compatible with tensorflow backend
'''
def focal_loss(gamma=2, alpha=0.75):
    def focal_loss_fixed(y_true, y_pred):#with tensorflow
        eps = 1e-12
        y_pred=K.clip(y_pred,eps,1.-eps)#improve the stability of the focal loss and see issues 1 for more information
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

class nn_feat_fusion():
    def __init__(self, idim, optimizer='Adadelta'):
        inputs = Input(shape=(idim,), name='input')
        layer1 = Dense(256)(inputs)
        layer2 = Activation('relu', name='representation')(layer1)
        #layer22 = Dense(256)(layer2)
        #layer23 = Activation('sigmoid', name='representation')(layer22)
        layer3 = Dense(4)(layer2)
        layer4 = Activation('softmax')(layer3)
        self.model = Model(inputs, layer4)
        self.rep = Model(self.model.inputs, self.model.get_layer('representation').output)
    
    def train(self, X, y):
        self.model.compile(optimizer='adam', loss='categorical_crossentropy')
        history = self.model.fit(X, y, validation_split=0.1, nb_epoch=2, shuffle=True)
        self.rep = Model(self.model.inputs, self.model.get_layer('representation').output)
        return history
    
    def feature(self, X):
        return self.rep.predict(X, verbose=0)
    
    def evaluate(self, X, y):
        return self.model.evaluate(X, y, verbose=0)

In [11]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Activation
import numpy as np

with open('cat_feat_mfcc_100_256.pkl') as f:
    X = pickle.load(f)
tr_x = X['feat']['train']
te_x = X['feat']['test']
tr_y = X['cat']['train']
te_y = X['cat']['test']

with open('feat/80_20/voiceQuality.pkl') as f:
    X = pickle.load(f)
tr_x = np.concatenate((tr_x, X['feat']['train']), axis=1)
te_x = np.concatenate((te_x, X['feat']['test']), axis=1)

with open('cat_feat_lpcc_80_256.pkl') as f:
    X = pickle.load(f)
tr_x = np.concatenate((tr_x, X['feat']['train']), axis=1)
te_x = np.concatenate((te_x, X['feat']['test']), axis=1)

fusion_nn = True
if fusion_nn:
    from sklearn.preprocessing import label_binarize
    tr_yc = label_binarize(tr_y, list(set([i for i in tr_y])))
    te_yc = label_binarize(te_y, list(set([i for i in te_y])))
    
    nn_comb = nn_feat_fusion(tr_x.shape[1])
    print nn_comb.train(tr_x, tr_yc)
    tr_x = nn_comb.feature(tr_x)
    te_x = nn_comb.feature(te_x)

with open('combined_256_mfcc_VQ_lpcc.pkl', 'wb') as f:
    pickle.dump({'feat':{'train':tr_x, 'test':te_x}, 'cat':{'train':tr_y, 'test':te_y}}, f)

clf = svm.SVC(kernel='rbf')
clf.fit(tr_x, tr_y)
pred = clf.predict(te_x)
print accuracy_score(te_y, pred)

Train on 3553 samples, validate on 395 samples
Epoch 1/2
3553/3553 [==============================] - 0s - loss: 0.7432 - val_loss: 1.2639
Epoch 2/2
3553/3553 [==============================] - 0s - loss: 0.6963 - val_loss: 1.3704
0.5425101214574899


In [118]:
from sklearn.metrics import confusion_matrix
cnf = confusion_matrix(te_y, pred)
print cnf

[[ 87  36  31   7]
 [ 47  72 121  18]
 [ 20  20 257  68]
 [  5   6  61 132]]


In [10]:
for i in range(len(list(pred == te_y))):
    print str(list(pred==te_y)[i]) + ' ' + str(pred[i]) + ' ' + str(te_y[i])

False 4 3
True 4 4
False 4 3
False 1 3
False 2 3
False 3 2
False 2 3
True 3 3
True 3 3
False 3 2
True 3 3
False 3 1
False 3 1
False 4 3
True 2 2
True 4 4
False 2 3
False 2 3
True 2 2
True 4 4
False 3 4
True 4 4
False 1 4
False 3 2
True 4 4
True 1 1
True 1 1
False 3 1
False 3 1
True 4 4
True 2 2
False 4 1
False 2 1
False 2 1
False 3 2
False 2 1
False 3 2
False 4 2
True 1 1
False 4 3
True 1 1
True 1 1
False 4 3
True 3 3
False 3 4
True 4 4
True 3 3
True 3 3
True 3 3
False 4 3
False 3 2
True 3 3
True 3 3
False 3 4
True 3 3
True 3 3
True 3 3
False 2 1
False 2 1
False 2 3
False 3 2
False 3 2
False 2 1
True 2 2
False 3 2
False 3 2
False 3 2
False 1 2
False 3 1
True 1 1
False 4 3
False 4 3
True 1 1
False 3 4
True 4 4
True 3 3
False 3 4
True 1 1
True 3 3
True 1 1
False 2 3
False 4 2
False 3 2
False 1 2
True 4 4
False 3 4
True 3 3
True 3 3
True 3 3
True 1 1
True 1 1
False 3 4
False 2 4
False 3 4
True 4 4
True 1 1
True 4 4
True 4 4
True 4 4
True 2 2
True 1 1
True 4 4
True 4 4
True 4 4
False 3 4
F